# Problem Solving Agents: Uninformed Search

## Problem-Solving Agents & State Space

When the correct action to take is not immediately obvious, an agent may need to to plan ahead: to consider a sequence of actions that form a path to a goal state.

Such an agent is called a **problem-solving agent**, and the computational process it undertakes is called **search**.

Problem-solving agents use atomic representations - states of the world are considered as wholes, with no internal structure visible to the problem-solving algorithms

We are going to build the **goal-based agents** that can plan ahead to solve problems.

In particular, we examine n*avigation problem/ route finding problem*.

We consider only the simplest environments: single agent, fully observable, deterministic, static, discrete, and known

1. We must begin by **precisely defining problems** -> `class Problem` - how we define a Problem in general 

 **!!!** Explore the `class Problem`

2. `class Problem` == The *abstract class* for a formal problem. 
* You should subclass     this and implement the methods `actions` and `result`, and possibly  ` __init__, goal_test`, and `path_cost`. 
* The *state space* should be included in a subclass
* Then you will create instances of your subclass and solve them with the various search functions.